In [ ]:
pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmp448oulw4".


In [ ]:
# @title
%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#define N 1024
#define MAX 9
#define P 12

__global__ void scan0(int* X, int* Y, int InputSize) {
	__shared__ int XY[N];
	int i = blockIdx.x * blockDim.x + threadIdx.x;
	Y[i] = 0;
	for (int j = 0; j <= i; j++) {
		Y[i] += X[j];
	}
}

__global__ void scan1(int* X, int* Y, int InputSize) {
	__shared__ int XY[N];
	int i = blockIdx.x * blockDim.x + threadIdx.x;
	Y[i] = 0;
	for (int j = 0; j <= i; j++) {
		Y[i] += X[j];
	}
}

__global__ void scan2(int* X, int* Y, int InputSize) {
	__shared__ int XY[N];
	int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < InputSize) {
		XY[threadIdx.x] = X[i];
	}
	for (int j = 1; j < blockDim.x; j *= 2) {
		__syncthreads();
		if (threadIdx.x >= j) {
			XY[threadIdx.x] += XY[threadIdx.x - j];
		}
		Y[i] = XY[threadIdx.x];
	}
}

__global__ void scan3(int* X, int* Y, int InputSize) {
	__shared__ int XY[N];
	int i = 2 * blockIdx.x * blockDim.x + threadIdx.x;
	if (i < InputSize) {
		XY[threadIdx.x] = X[i];
	}
	if (i + blockDim.x < InputSize) {
		XY[threadIdx.x + blockDim.x] = X[i + blockDim.x];
	}

	for (int j = 1; j <= blockDim.x; j *= 2) {
		__syncthreads();
		int index = (threadIdx.x + 1) * 2 * j - 1;
		if (index < N) {
			XY[index] += XY[index - j];
		}
	}

	for (int j = N / 4; j > 0; j /= 2) {
		__syncthreads();
		int index = (threadIdx.x + 1) * j * 2 - 1;
		if (index + j < N) {
			XY[index + j] += XY[index];
		}
	}

	__syncthreads();
	if (i < InputSize) {
		Y[i] = XY[threadIdx.x];
	}
	if (i + blockDim.x < InputSize) {
		Y[i + blockDim.x] = XY[threadIdx.x + blockDim.x];
	}
}

int main(int argc, char** argv) {

	int* data, * h_obstacle, * h_prefixSum; //Host pointers
	int* d_obstacle, * d_prefixSum0, * d_prefixSum1, * d_prefixSum2, * d_prefixSum3; //Device pointers

	//Allocate and initialize matrices
	data = (int*)malloc(N * sizeof(int));
	h_obstacle = (int*)malloc(N * sizeof(int));
	h_prefixSum = (int*)malloc(N * sizeof(int));

	//Populate with distances
	for (int i = 0; i < N; i++) {
		int j = 8;
		if (i % j == 0) {
			data[i] = 3;
			//if (i+1 <N) { //this is used to make the obstacles wider in the array.
          //data[i+1]=3;
          //i++;
      //}
		}
		else {
			data[i] = 9;
		}
	}

	//Check if obstacle detected
	for (int i = 0; i < N; i++) {
		if (data[i] < MAX) {
			h_obstacle[i] = 1;
		}
		else {
			h_obstacle[i] = 0;
		}
	}

	//@@Display result
  printf("Array Size = %d\n",N);
	printf("\nData Array:\n");
 	for (int i = 0; i < P; i++) {
		printf("%d   ", data[i]);
	}
  printf(" . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("   %d", data[i]);
	}

	printf("\nObstacle Array:\n");
	for (int i = 0; i < P; i++) {
		printf("%d   ", h_obstacle[i]);
	}
  printf(" . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("   %d", h_obstacle[i]);
	}



	//@@Allocate GPU Memory
	cudaMalloc((void**)&d_obstacle, N * sizeof(int));
	cudaMalloc((void**)&d_prefixSum0, N * sizeof(int));
	cudaMalloc((void**)&d_prefixSum1, N * sizeof(int));
	cudaMalloc((void**)&d_prefixSum2, N * sizeof(int));
	cudaMalloc((void**)&d_prefixSum3, N * sizeof(int));

	//@@Copy memory to GPU
	cudaMemcpy(d_obstacle, h_obstacle, N * sizeof(int), cudaMemcpyHostToDevice);

	//@@Initialize the grid and block dimensions here
	dim3 blockSize(N);
	dim3 gridSize((int)ceil((float)N / blockSize.x));

	cudaEvent_t start0, stop0, start1, stop1, start2, stop2, start3, stop3;
	cudaEventCreate(&start0);
	cudaEventCreate(&stop0);
	float t0 = 0;
 	cudaEventCreate(&start1);
	cudaEventCreate(&stop1);
	float t1 = 0;
	cudaEventCreate(&start2);
	cudaEventCreate(&stop2);
	float t2 = 0;
	cudaEventCreate(&start3);
	cudaEventCreate(&stop3);
	float t3 = 0;

		//@@Launch GPU kernel
	cudaEventRecord(start0);
	scan1 << <gridSize, blockSize >> > (d_obstacle, d_prefixSum0, N);
	cudaDeviceSynchronize();
	cudaEventRecord(stop0);

	//@@Copy GPU memory back to CPU
	cudaMemcpy(h_prefixSum, d_prefixSum0, N * sizeof(int), cudaMemcpyDeviceToHost);

	cudaEventSynchronize(stop0);
	cudaEventElapsedTime(&t0, start0, stop0);
	printf("\n\nTime (Brute-Force): %f ms", t0);

	printf("\nPrefix Sum Array:\n");
	for (int i = 0; i < P; i++) {
		printf("%4.0d", h_prefixSum[i]);
	}
  printf("  . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("%4.0d", h_prefixSum[i]);
	}
	printf("\n%4.0d obstacles detected", h_prefixSum[N - 1]);

	//@@Launch GPU kernel
	cudaEventRecord(start1);
	scan1 << <gridSize, blockSize >> > (d_obstacle, d_prefixSum1, N);
	cudaDeviceSynchronize();
	cudaEventRecord(stop1);

	//@@Copy GPU memory back to CPU
	cudaMemcpy(h_prefixSum, d_prefixSum1, N * sizeof(int), cudaMemcpyDeviceToHost);

	cudaEventSynchronize(stop1);
	cudaEventElapsedTime(&t1, start1, stop1);
	printf("\n\nTime (Brute-Force): %f ms", t1);

	printf("\nPrefix Sum Array:\n");
	for (int i = 0; i < P; i++) {
		printf("%4.0d", h_prefixSum[i]);
	}
  printf("  . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("%4.0d", h_prefixSum[i]);
	}
	printf("\n%4.0d obstacles detected", h_prefixSum[N - 1]);

	//@@Launch GPU kernel
	cudaEventRecord(start2);
	scan2 << <gridSize, blockSize >> > (d_obstacle, d_prefixSum2, N);
	cudaDeviceSynchronize();
	cudaEventRecord(stop2);

	//@@Copy GPU memory back to CPU
	cudaMemcpy(h_prefixSum, d_prefixSum2, N * sizeof(int), cudaMemcpyDeviceToHost);

	cudaEventSynchronize(stop2);
	cudaEventElapsedTime(&t2, start2, stop2);
	printf("\n\nTime (Kogge-Stone): %f ms", t2);

	printf("\nPrefix Sum Array:\n");
	for (int i = 0; i < P; i++) {
		printf("%4.0d", h_prefixSum[i]);
	}
  printf("  . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("%4.0d", h_prefixSum[i]);
	}
	printf("\n%d obstacles detected", h_prefixSum[N - 1]);

	//@@Launch GPU kernel
	cudaEventRecord(start3);
	scan3 << <gridSize, blockSize >> > (d_obstacle, d_prefixSum3, N);
	cudaDeviceSynchronize();
	cudaEventRecord(stop3);

	//@@Copy GPU memory back to CPU
	cudaMemcpy(h_prefixSum, d_prefixSum3, N * sizeof(int), cudaMemcpyDeviceToHost);

	cudaEventSynchronize(stop3);
	cudaEventElapsedTime(&t3, start3, stop3);
	printf("\n\nTime (Brent-Kung): %f ms", t3);

	printf("\nPrefix Sum Array:\n");
	for (int i = 0; i < P; i++) {
		printf("%4.0d", h_prefixSum[i]);
	}
  printf("  . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("%4.0d", h_prefixSum[i]);
	}
	printf("\n%d obstacles detected", h_prefixSum[N - 1]);

	//@@Free GPU memory
	cudaFree(d_obstacle);
	cudaFree(d_prefixSum0);
	cudaFree(d_prefixSum1);
	cudaFree(d_prefixSum2);
	cudaFree(d_prefixSum3);

	//@@Free host memory
	free(h_obstacle);
	free(h_prefixSum);
}

Array Size = 1024

Data Array:
3   9   9   9   9   9   9   9   3   9   9   9    . . .    9   9   9   9   3   9   9   9   9   9   9   9
Obstacle Array:
1   0   0   0   0   0   0   0   1   0   0   0    . . .    0   0   0   0   1   0   0   0   0   0   0   0

Time (Brute-Force): 0.438144 ms
Prefix Sum Array:
   1   1   1   1   1   1   1   1   2   2   2   2  . . .  127 127 127 127 128 128 128 128 128 128 128 128
 128 obstacles detected

Time (Brute-Force): 0.174464 ms
Prefix Sum Array:
   1   1   1   1   1   1   1   1   2   2   2   2  . . .  127 127 127 127 128 128 128 128 128 128 128 128
 128 obstacles detected

Time (Kogge-Stone): 0.030432 ms
Prefix Sum Array:
   1   1   1   1   1   1   1   1   2   2   2   2  . . .  163 163 163 163 166 166 166 166 164 164 164 164
164 obstacles detected

Time (Brent-Kung): 0.036096 ms
Prefix Sum Array:
   1   1   1   1   1   1   1   1   2   2   2   2  . . .  127 127 127 127 128 128 128 128 128 128 128 128
128 obstacles detected


In [ ]:
# @title
%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#define N 1024
#define MAX 9
#define P 12


__global__ void scan1(int* X, int* Y, int InputSize) {
	__shared__ int XY[N];
	int i = blockIdx.x * blockDim.x + threadIdx.x;
	Y[i] = 0;
	for (int j = 0; j <= i; j++) {
		Y[i] += X[j];
	}
}


int main(int argc, char** argv) {

	int* data, * h_obstacle, * h_prefixSum; //Host pointers
	int* d_obstacle, * d_prefixSum1; //Device pointers

	//Allocate and initialize matrices
	data = (int*)malloc(N * sizeof(int));
	h_obstacle = (int*)malloc(N * sizeof(int));
	h_prefixSum = (int*)malloc(N * sizeof(int));

	//Populate with distances
	for (int i = 0; i < N; i++) {
		int j = 8;
		if (i % j == 0) {
			data[i] = 3;
			//if (i+1 <N) { //this is used to make the obstacles wider in the array.
          //data[i+1]=3;
          //i++;
      //}
		}
		else {
			data[i] = 9;
		}
	}

	//Check if obstacle detected
	for (int i = 0; i < N; i++) {
		if (data[i] < MAX) {
			h_obstacle[i] = 1;
		}
		else {
			h_obstacle[i] = 0;
		}
	}

	//@@Allocate GPU Memory
	cudaMalloc((void**)&d_obstacle, N * sizeof(int));
	cudaMalloc((void**)&d_prefixSum1, N * sizeof(int));

	//@@Copy memory to GPU
	cudaMemcpy(d_obstacle, h_obstacle, N * sizeof(int), cudaMemcpyHostToDevice);

	//@@Initialize the grid and block dimensions here
	dim3 blockSize(N/16);
	dim3 gridSize((int)ceil((float)N / blockSize.x));

 	//@@Display result
  printf("Array Size = %d\n",N);
  printf("Block Size = %d\n",blockSize);
  printf("Grid Size = %d\n",gridSize);
	printf("\nData Array:\n");
 	for (int i = 0; i < P; i++) {
		printf("%d  ", data[i]);
	}
  printf(" . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("  %d", data[i]);
	}

	printf("\nObstacle Array:\n");
	for (int i = 0; i < P; i++) {
		printf("%d  ", h_obstacle[i]);
	}
  printf(" . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("  %d", h_obstacle[i]);
	}

	cudaEvent_t start1, stop1;
	cudaEventCreate(&start1);
	cudaEventCreate(&stop1);
	float t1 = 0;


	//@@Launch GPU kernel
	cudaEventRecord(start1);
	scan1 << <gridSize, blockSize >> > (d_obstacle, d_prefixSum1, N);
	cudaDeviceSynchronize();
	cudaEventRecord(stop1);

	//@@Copy GPU memory back to CPU
	cudaMemcpy(h_prefixSum, d_prefixSum1, N * sizeof(int), cudaMemcpyDeviceToHost);

	cudaEventSynchronize(stop1);
	cudaEventElapsedTime(&t1, start1, stop1);
	printf("\n\nTime (Brute-Force): %f ms", t1);

	printf("\nPrefix Sum Array:\n");
	for (int i = 0; i < P; i++) {
		printf("%3.0d", h_prefixSum[i]);
	}
  printf("  . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("%3.0d", h_prefixSum[i]);
	}
	printf("\n%3.0d obstacles detected", h_prefixSum[N - 1]);


	//@@Free GPU memory
	cudaFree(d_obstacle);
	cudaFree(d_prefixSum1);

	//@@Free host memory
	free(h_obstacle);
	free(h_prefixSum);
}

Array Size = 1024
Block Size = 64
Grid Size = 16

Data Array:
3  9  9  9  9  9  9  9  3  9  9  9   . . .   9  9  9  9  3  9  9  9  9  9  9  9
Obstacle Array:
1  0  0  0  0  0  0  0  1  0  0  0   . . .   0  0  0  0  1  0  0  0  0  0  0  0

Time (Brute-Force): 0.286560 ms
Prefix Sum Array:
  1  1  1  1  1  1  1  1  2  2  2  2  . . . 127127127127128128128128128128128128
128 obstacles detected


In [ ]:
# @title

%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#define N 1024
#define MAX 9
#define P 12



__global__ void scan2(int* X, int* Y, int InputSize) {
	__shared__ int XY[N];
	int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < InputSize) {
		XY[threadIdx.x] = X[i];
	}
	for (int j = 1; j < blockDim.x; j *= 2) {
		__syncthreads();
		if (threadIdx.x >= j) {
			XY[threadIdx.x] += XY[threadIdx.x - j];
		}
		Y[i] = XY[threadIdx.x];
	}
}


int main(int argc, char** argv) {

	int* data, * h_obstacle, * h_prefixSum; //Host pointers
	int* d_obstacle, * d_prefixSum2; //Device pointers

	//Allocate and initialize matrices
	data = (int*)malloc(N * sizeof(int));
	h_obstacle = (int*)malloc(N * sizeof(int));
	h_prefixSum = (int*)malloc(N * sizeof(int));

	//Populate with distances
	for (int i = 0; i < N; i++) {
		int j = 8;
		if (i % j == 0) {
			data[i] = 3;
			//if (i+1 <N) { //this is used to make the obstacles wider in the array.
          //data[i+1]=3;
          //i++;
      //}
		}
		else {
			data[i] = 9;
		}
	}

	//Check if obstacle detected
	for (int i = 0; i < N; i++) {
		if (data[i] < MAX) {
			h_obstacle[i] = 1;
		}
		else {
			h_obstacle[i] = 0;
		}
	}

	//@@Display result
  printf("Array Size = %d\n",N);
	printf("\nData Array:\n");
 	for (int i = 0; i < P; i++) {
		printf("%d  ", data[i]);
	}
  printf(" . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("  %d", data[i]);
	}

	printf("\nObstacle Array:\n");
	for (int i = 0; i < P; i++) {
		printf("%d  ", h_obstacle[i]);
	}
  printf(" . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("  %d", h_obstacle[i]);
	}



	//@@Allocate GPU Memory
	cudaMalloc((void**)&d_obstacle, N * sizeof(int));

	cudaMalloc((void**)&d_prefixSum2, N * sizeof(int));


	//@@Copy memory to GPU
	cudaMemcpy(d_obstacle, h_obstacle, N * sizeof(int), cudaMemcpyHostToDevice);

	//@@Initialize the grid and block dimensions here
	dim3 blockSize(N);
	dim3 gridSize((int)ceil((float)N / blockSize.x));

	cudaEvent_t start2, stop2;

	cudaEventCreate(&start2);
	cudaEventCreate(&stop2);
	float t2 = 0;

	//@@Launch GPU kernel
	cudaEventRecord(start2);
	scan2 << <gridSize, blockSize >> > (d_obstacle, d_prefixSum2, N);
	cudaDeviceSynchronize();
	cudaEventRecord(stop2);

	//@@Copy GPU memory back to CPU
	cudaMemcpy(h_prefixSum, d_prefixSum2, N * sizeof(int), cudaMemcpyDeviceToHost);

	cudaEventSynchronize(stop2);
	cudaEventElapsedTime(&t2, start2, stop2);
	printf("\n\nTime (Kogge-Stone): %f ms", t2);

	printf("\nPrefix Sum Array:\n");
	for (int i = 0; i < P; i++) {
		printf("%3.0d", h_prefixSum[i]);
	}
  printf("  . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("%3.0d", h_prefixSum[i]);
	}
	printf("\n%d obstacles detected", h_prefixSum[N - 1]);

	//@@Free GPU memory
	cudaFree(d_obstacle);
	cudaFree(d_prefixSum2);

	//@@Free host memory
	free(h_obstacle);
	free(h_prefixSum);
}

Array Size = 1024

Data Array:
3  9  9  9  9  9  9  9  3  9  9  9   . . .   9  9  9  9  3  9  9  9  9  9  9  9
Obstacle Array:
1  0  0  0  0  0  0  0  1  0  0  0   . . .   0  0  0  0  1  0  0  0  0  0  0  0

Time (Kogge-Stone): 0.174240 ms
Prefix Sum Array:
  1  1  1  1  1  1  1  1  2  2  2  2  . . . 127127127127128128128128128128128128
128 obstacles detected


In [ ]:
# @title

%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#define N 1024
#define MAX 9
#define P 12


__global__ void scan3(int* X, int* Y, int InputSize) {
	__shared__ int XY[N];
	int i = 2 * blockIdx.x * blockDim.x + threadIdx.x;
	if (i < InputSize) {
		XY[threadIdx.x] = X[i];
	}
	if (i + blockDim.x < InputSize) {
		XY[threadIdx.x + blockDim.x] = X[i + blockDim.x];
	}

	for (int j = 1; j <= blockDim.x; j *= 2) {
		__syncthreads();
		int index = (threadIdx.x + 1) * 2 * j - 1;
		if (index < N) {
			XY[index] += XY[index - j];
		}
	}

	for (int j = N / 4; j > 0; j /= 2) {
		__syncthreads();
		int index = (threadIdx.x + 1) * j * 2 - 1;
		if (index + j < N) {
			XY[index + j] += XY[index];
		}
	}

	__syncthreads();
	if (i < InputSize) {
		Y[i] = XY[threadIdx.x];
	}
	if (i + blockDim.x < InputSize) {
		Y[i + blockDim.x] = XY[threadIdx.x + blockDim.x];
	}
}

int main(int argc, char** argv) {

	int* data, * h_obstacle, * h_prefixSum; //Host pointers
	int* d_obstacle, * d_prefixSum3; //Device pointers

	//Allocate and initialize matrices
	data = (int*)malloc(N * sizeof(int));
	h_obstacle = (int*)malloc(N * sizeof(int));
	h_prefixSum = (int*)malloc(N * sizeof(int));

	//Populate with distances
	for (int i = 0; i < N; i++) {
		int j = 8;
		if (i % j == 0) {
			data[i] = 3;
			//if (i+1 <N) { //this is used to make the obstacles wider in the array.
          //data[i+1]=3;
          //i++;
      //}
		}
		else {
			data[i] = 9;
		}
	}

	//Check if obstacle detected
	for (int i = 0; i < N; i++) {
		if (data[i] < MAX) {
			h_obstacle[i] = 1;
		}
		else {
			h_obstacle[i] = 0;
		}
	}

	//@@Display result
  printf("Array Size = %d\n",N);
	printf("\nData Array:\n");
 	for (int i = 0; i < P; i++) {
		printf("%d  ", data[i]);
	}
  printf(" . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("  %d", data[i]);
	}

	printf("\nObstacle Array:\n");
	for (int i = 0; i < P; i++) {
		printf("%d  ", h_obstacle[i]);
	}
  printf(" . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("  %d", h_obstacle[i]);
	}



	//@@Allocate GPU Memory
	cudaMalloc((void**)&d_obstacle, N * sizeof(int));
	cudaMalloc((void**)&d_prefixSum3, N * sizeof(int));

	//@@Copy memory to GPU
	cudaMemcpy(d_obstacle, h_obstacle, N * sizeof(int), cudaMemcpyHostToDevice);

	//@@Initialize the grid and block dimensions here
	dim3 blockSize(N);
	dim3 gridSize((int)ceil((float)N / blockSize.x));

	cudaEvent_t start3, stop3;
	cudaEventCreate(&start3);
	cudaEventCreate(&stop3);
	float t3 = 0;


	//@@Launch GPU kernel
	cudaEventRecord(start3);
	scan3 << <gridSize, blockSize >> > (d_obstacle, d_prefixSum3, N);
	cudaDeviceSynchronize();
	cudaEventRecord(stop3);

	//@@Copy GPU memory back to CPU
	cudaMemcpy(h_prefixSum, d_prefixSum3, N * sizeof(int), cudaMemcpyDeviceToHost);

	cudaEventSynchronize(stop3);
	cudaEventElapsedTime(&t3, start3, stop3);
	printf("\n\nTime (Brent-Kung): %f ms", t3);

	printf("\nPrefix Sum Array:\n");
	for (int i = 0; i < P; i++) {
		printf("%3.0d", h_prefixSum[i]);
	}
  printf("  . . . ");
 	for (int i = N-P; i < N; i++) {
		printf("%3.0d", h_prefixSum[i]);
	}
	printf("\n%d obstacles detected", h_prefixSum[N - 1]);

	//@@Free GPU memory
	cudaFree(d_obstacle);
	cudaFree(d_prefixSum3);

	//@@Free host memory
	free(h_obstacle);
	free(h_prefixSum);
}

Array Size = 1024

Data Array:
3  9  9  9  9  9  9  9  3  9  9  9   . . .   9  9  9  9  3  9  9  9  9  9  9  9
Obstacle Array:
1  0  0  0  0  0  0  0  1  0  0  0   . . .   0  0  0  0  1  0  0  0  0  0  0  0

Time (Brent-Kung): 0.194912 ms
Prefix Sum Array:
  1  1  1  1  1  1  1  1  2  2  2  2  . . . 127127127127128128128128128128128128
128 obstacles detected
